# Demo 3 - Hadoop Shuffle & TOS
__`MIDS w261: Machine Learning at Scale | UC Berkeley School of Information | Spring 2019`__

Designing MapReduce algorithms involves two kinds of planning. First we have to figure out which parts of a calculation can be performed in parallel and which can't. Then we have to figure out how to put those pieces together so that the right information ends up in the right place at the right time. That's what the Hadoop shuffle is all about. Today we'll talk about a few techniques to optimize your Hadoop jobs. By the end of this demo you should be able to:  
* ... __identify__ what makes the Hadoop Shuffle potentially costly.
* ... __define__ local aggregation & identify when it won't help.
* ... __implement__ partial, unordered, and total order sort.
* ... __create__ custom counters for your Hadoop Jobs.
* ... __describe__ the order inversion pattern & when to use it (ie: relative frequencies).

**Note**: Hadoop Streaming syntax is very particular. Make sure to test your python scripts before passing them to the Hadoop job and pay careful attention to the order in which Hadoop job parameters are specified.

### Notebook Set-Up

In [ ]:
# imports & magic commands
import sys
%reload_ext autoreload
%autoreload 2

In [ ]:
# globals
JAR_FILE = "/usr/lib/hadoop-mapreduce/hadoop-streaming.jar"
HOME_DIR = "/media/notebooks" # this is where docker mounts your repo, ADJUST AS NEEDED
DEMO_DIR = HOME_DIR + "/LiveSessionMaterials/wk03Demo_HadoopShuffle"
HDFS_DIR = "/user/root/demo3"
!hdfs dfs -mkdir {HDFS_DIR}

In [ ]:
# get path to notebook
PWD = !pwd
PWD = PWD[0]

In [ ]:
# store notebook environment path
from os import environ
PATH  = environ['PATH']

__`REMINDER:`__ If you are running this notebook from the course Docker container you can track your Hadoop Jobs using the UI at: http://localhost:19888/jobhistory/

### Load the Data

In this notebook, we'll continue working with the  _Alice in Wonderland_ text file from HW1 and the test file we created for debugging. Run the following cell to confirm that you have access to these files and save their location to a global variable to use in your Hadoop Streaming jobs. 

In [ ]:
# make a data subfolder - RUN THIS CELL AS IS
!mkdir -p data

In [ ]:
# (Re)Download Alice Full text from Project Gutenberg - RUN THIS CELL AS IS 
# NOTE: feel free to replace 'curl' with 'wget' or equivalent command of your choice.
!curl "http://www.gutenberg.org/files/11/11-0.txt" -o data/alice.txt

In [ ]:
%%writefile data/alice_test.txt
This is a small test file. This file is for a test.
This small test file has two small lines.

In [ ]:
# save the paths - RUN THIS CELL AS IS (if Option 1 failed)
ALICE_TXT = PWD + "/data/alice.txt"
TEST_TXT = PWD + "/data/alice_test.txt"

In [ ]:
# confirm the files are there - RUN THIS CELL AS IS
!echo "######### alice.txt #########"
!head -n 6 {ALICE_TXT}
!echo "######### alice_test.txt #########"
!cat {TEST_TXT}

__Load the file into HDFS for easy access__

In [ ]:
# load the input files into HDFS (RUN THIS CELL AS IS)
!hdfs dfs -copyFromLocal {TEST_TXT} {HDFS_DIR}
!hdfs dfs -copyFromLocal {ALICE_TXT} {HDFS_DIR}

# Content Overview: Hadoop Shuffle

The week 3 reading from Chapter 3 of _Data Intensive Text Processing with Map Reduce_ by Lin and Dyer discusses 5 key techniques for controlling execution and managing the flow of data in MapReduce:
1. using complex data structures to communicate partial results
2. user-specified initialization/termination code before/after each map/reduce task
3. preserving state across multiple keys
4. controling the sort order of intermediate keys
5. specifying the partitioning of the key space

### Our goal in employing these techniques is to minimize the amount we have to move the data. This involves keeping track of what data is stored where at each stage in our job (_DITP figure 2.5 and 2.6_):
*Recall that a Hadoop cluster stores our data on datanotes and ships the programmer's map and reduce code to those nodes to perform those transformations in place.*

![HDFSdiagram](DITP_fig2-5,6.png)


### Though it isn't always possible to do so, ideally we'd like to design an implementation that retrieves the result in a single MapReduce job (_DITP figure 2.4_):
![MRdiagram](DITP_fig2-4.png)

### Shuffle & Sort Detail (_Hadoop, The Definitve Guide, by Tom White; fig 7-4_):   
![CircularBuffer-DFG](HDG_fig7-4-annotated.png)


> __DISCUSSION QUESTIONS:__  
> * What work does your Hadoop cluster have to do at the shuffle stage? 
> * What determines the time complexity of this work? 
> * Compare the Lyn & Dyer diagram with the om White diagram. How might the Lyn & Dyer diagram be misleading?
> * What is a combiner, how does it impact the shuffle? 'where' does the combining happen?
> * What is local aggregation? Is a combiner the only way to do it?
> * What is a partitioner and how does it impact the shuffle? How/where can we specify custom paritioning behavior? If we don't specify a partitioner, will Hadoop still partition the data? How?
> * Can you think of an example of a task that can't be accomplished in a single MapReduce job? Explain.

#### *_A note on the Lin and Dyer reading (and other readings you may encounter):_

>In MapReduce, the programmer defines a mapper and a reducer with the following signatures:   
map: (k1; v1) -> [(k2; v2)]   
reduce: (k2; [v2]) -> [(k3; v3)]   
where [..] denotes a list

_It is important to note that the MapReduce framework ensures that the keys are ordered, so we know that if a key is different from the previous one, we have moved into a new key group. In contrast to the Java API, where you are provided an iterator over each key group (as per above Lin and Dyer version), in Streaming you have to find key group boundaries in your program. (Page 39, Hadoop - The Definitive Guide)_

# Exercise 1: Relative Frequencies
In last week's live session and HW1 we used Word Count as a cannonical example of an embarassingly parallel task. At first glance, computing relative frequencies (`word count / total count`) seems like it would be just as easy to implement -- after all it's just word count with an extra division at the end. However this task actually presents a small design challenge that is perfect to illustrate a few of the techniques that Lin & Dyer talk about.

__DISCUSSION:__
> * Talk through what the MapReduce job would look like? What is the challenge here?
> * Is it possible to compute relative frequencies in a single MapReduce job? 
> * Is it possible to compute relative frequencies in a single MapReduce job with multiple reducers?

### Exercise 1 Tasks:

* __a) read provided code:__ We've provided a naive interpretation to get you started. Take a look at __`Frequencies/mapper.py`__, __`Frequencies/combiner.py`__  and __`Frequencies/reducer.py`__.  

* __b) discuss:__ How does it resolve the challenge of computing the total? Uncomment line 22 in the mapper and lines 26-27 in the reducer to take advantage of this 'solution', then run the unit tests below, to confirm that you understand what's going on in each of these scripts. Despite solving the problem of computing the total in a single map-reduce job, what is wrong with this approach? 

* __c) fix the problem:__ To fix the problem, all we need to do is make a small change to the key used to emit the total counts (you need to do this in both the mapper and reducer). Think about Hadoop's default sorting. What keys arrive first? What key could you assign to the total that would be guaranteed to arrive first? Once you are satisfied that your solution works, run the provided Hadoop job on the test file & then the full `alice` text. (note it has a single reducer for now).  

* __d) discuss:__  Now, re-run the job with 4 reducers, what happens to the results? What would we have to do to fix the problem?

In [ ]:
# part b - make sure scripts are executable (RUN THIS CELL AS IS)
!chmod a+x Frequencies/mapper.py
!chmod a+x Frequencies/combiner.py
!chmod a+x Frequencies/reducer.py

In [ ]:
# part b - unit test mapper script
!echo "foo foo quux labs foo bar quux" | Frequencies/mapper.py

In [ ]:
# part b - unit test map-combine (sort mimics shuffle) (RUN THIS CELL AS IS)
!echo "foo foo quux labs foo bar quux" | Frequencies/mapper.py | sort -k1,1 | Frequencies/combiner.py

In [ ]:
# part b - unit test map-combine-reduce (sort mimics shuffle) (RUN THIS CELL AS IS)
!echo "foo foo quux labs foo bar quux" | Frequencies/mapper.py | sort -k1,1 | Frequencies/combiner.py | Frequencies/reducer.py 

In [ ]:
# parts c - clear the output directory (RUN THIS CELL AS IS)
!hdfs dfs -rm -r {HDFS_DIR}/frequencies-output
# NOTE: this directory won't exist unless you are re-running a job, that's fine.

In [ ]:
# parts c - Hadoop streaming job (RUN THIS CELL AS IS FIRST, then make your modification)
!hadoop jar {JAR_FILE} \
  -files Frequencies/reducer.py,Frequencies/mapper.py,Frequencies/combiner.py \
  -mapper mapper.py \
  -combiner combiner.py \
  -reducer reducer.py \
  -input {HDFS_DIR}/alice_test.txt \
  -output {HDFS_DIR}/frequencies-output \
  -cmdenv PATH={PATH} \
  -numReduceTasks 1

In [ ]:
# take a look at the first few results (RUN THIS CELL AS IS)
!hdfs dfs -cat {HDFS_DIR}/frequencies-output/part-00000 | head -n 10

__Expected Results:__

<table>
<th>part c (test)</th>
<th>part c (full)</th>
<tr>
<td><pre>
!total	1.0
a	0.1
file	0.15
for	0.05
has	0.05
is	0.1
lines	0.05
small	0.15
test	0.15
this	0.15
</pre></td>
<td><pre>
!total	1.0
a	0.0226802090523617
abide	6.57397363836571e-05
able	3.286986819182855e-05
about	0.0033527265555665124
above	9.860960457548565e-05
absence	3.286986819182855e-05
absurd	6.57397363836571e-05
accept	3.286986819182855e-05
acceptance	3.286986819182855e-05
</pre></td>
</tr></table>


__DISCUSSION:__  
> *  Was the original implementation scalable? How does it resolve the challenge of computing the total? 
> * Despite solving that problem, what is wrong with this approach?  
> * Did you come up with a new key that is guaranteed to arrive first? how?
> * What happened when you went from 1 reducer to 4, why? (HINT: Use the UI to see the error logs for the failed reduce tasks)
> * What do we need to do with the total counts when we move up to 4 reducers?

# Exercise 2: Custom Partitioning

Last week in Breakout 4 you learned out to implement a secondary sort -- that is, you learned how to tell Hadoop to order key-value pairs within each partition based on the value. However you also saw the limitation of this simple secondary sort: namely that sorting within a partition is not very useful if you need to use multiple reducers because, for example, the top word could end up in any one of the partitions and the next highest might end up in a totally different partition. Of course post processing your partially sorted partition files (eg. using mergesort) might solve this problem, but if your data is too large to fit on a single machine that is not a viable solution.

Luckily, Hadoop provides a way to partition the data across reducers in a user-defined way.  This is done telling Hadoop to use all or part of the composite key as a partitioning key. All lines with the same partition key are guaranteed to be processed by the same reducer. This is similar to the sort key, but allows for control at a higher level. This "custom partitioning" will both solve our sorting troubles from last week and solve the problem you saw when using multiple reducers in Exercise 1 this week. To use custom partitioning, there are 2 more parameters we need to add to our Hadoop Streaming Command:

 __`-D mapreduce.partition.keypartitioner.options="-k1,1"`__: tells Hadoop that we want to use the first key field as a partition key. Note: just like the `keycomparator`, `keypartitioner` must be used in conjuction with `stream.num.map.output.key.fields`.

__`-partitioner org.apache.hadoop.mapred.lib.KeyFieldBasedPartitioner`__: tells Hadoop that we want to partition the data in a custom way. Note about partitioner: this line **MUST** appear after all of the `-D` options or it will be ignored.

> __DISCUSSION QUESTIONS (before exercise 2):__
* Quick review: What is a composite key? 
* Quick review: Practically speaking, what is a 'partition'? How is this concept related to the HDFS output of a job?
* What is the difference between the partition key & the 'sort' key?

### Exercise 2 Tasks:

* __a) read provided code:__ In __`PartitionSort/mapper.py`__ we've provided a function that will assign a custom partition key (just a letter) to each word. We're going to use this mapper to sort our sample file with 3 partitions. Read this script. 

* __b) discus:__ How does the mapper decide which partition to assign each record? When you print out the results in what order do you expect to see the records?

* __c) Hadoop job:__ Add the required parameters to complete the Hadoop Streaming code below. Your job should partition based on the newly added first key field, and sort alphabetically by word. Run your job. [__`Hint:`__ Don't forget to specify the number of fields!]

* __d) discuss:__ Examine the output from your job. Compare it to the partitioning function we used. Are the words sorted alphabetically? What is suprising about the partition key that ended up in `part-00000`?

* __e) code:__ If time permits, modify your job so that it sorts the words by _count_ instead (still using 3 partitions). To do this you will need to change the partition function in __`PartitionSort/mapper.py`__ so that it partitions based on count instead of the first letter of the word. Use 4 and 8 as your cut-points. You will also need to modify one of the Hadoop parameters (which one? why?). Run your job. Are you able to get a total sort? Why/why not?

In [ ]:
%%writefile PartitionSort/sample.txt
foo	5
quux	9
labs	100
bar	5
qi	1

In [ ]:
# load sample file into HDFS (RUN THIS CELL AS IS)
!hdfs dfs -copyFromLocal PartitionSort/sample.txt {HDFS_DIR}

In [ ]:
# part a - complete your work above then RUN THIS CELL AS IS
!chmod a+x PartitionSort/mapper.py

In [ ]:
# parts a - clear output directory (RUN THIS CELL AS IS)
!hdfs dfs -rm -r {HDFS_DIR}/psort-output

In [ ]:
# part a - Hadoop streaming command - ADD SORT and PARTITION PARAMETERS HERE
!hadoop jar {JAR_FILE} \




  -files PartitionSort/mapper.py \
  -mapper mapper.py \
  -reducer /bin/cat \

  -input {HDFS_DIR}/sample.txt \
  -output {HDFS_DIR}/psort-output \
  -cmdenv PATH={PATH}\
  -numReduceTasks 3

In [ ]:
# part a - Save results locally (RUN THIS CELL AS IS)
!hdfs dfs -cat {HDFS_DIR}/psort-output/part-0000* > PartitionSort/results.txt

In [ ]:
# part a - view results (RUN THIS CELL AS IS)
!head PartitionSort/results.txt

In [ ]:
# part a - look at first partition (RUN THIS CELL AS IS)
!hdfs dfs -cat {HDFS_DIR}/psort-output/part-00000

In [ ]:
# part a - look at second partition (RUN THIS CELL AS IS)
!hdfs dfs -cat {HDFS_DIR}/psort-output/part-00001

In [ ]:
# part a - look at third partition (RUN THIS CELL AS IS)
!hdfs dfs -cat {HDFS_DIR}/psort-output/part-00002

__Expected Output:__

<table>
<th>part c</th>
<th>part e</th>
<tr>
<td><pre>
B	labs	100
C	qi	1
C	quux	9
A	bar	5
A	foo	5
</pre></td>
<td><pre>
B	labs	100
B	quux	9
C	foo	5
C	bar	5
A	qi	1
</pre></td>
</tr></table>

> __DISCUSSION QUESTIONS (exercise 2 debrief):__
* In the provided implementation how did we assign records to partitions?
* In part c, why didn't this partitioning result in alphabetically sorted words?
* Given what you saw in part c, how did you 'trick' Hadoop into doing a full sort (by count) in part e?
* If you didn't achive the full sort in `e` why not? On a larger data set what postprocessing would you have to do in this kind of scenario? Is this postprocessing non-trivial?
* In addition to changing the partition function what other Hadoop parameter did you have to change for part `e`?
* In the real world you wouldn't want your partition key as part of your output. What would we do to avoid this?
* Does anyone need any additional clarification about any of the Hadoop Streaming?

# Exercise 3: Relative Frequencies Revisited
### (+ intro to counters)

Now that you know how to use custom partition keys, let's use this techinique to fix the problem from part d in Exercise 1.

First a brief digression... A common challenge when implementing custom partitioning at scale is load balancing: _how can we ensure that each reducer node is doing approximately the same amount of work?_ Hadoop's option to define custom counters can be a useful way to monitor this kind of detail. Just like the built in  counters that you learned about last week, custom counters are a slight departure from statelessness. Normally we wouldn't want to share a mutable variable across multiple nodes, however in this case the framework manages so that you can increment them from any node without causing a race condition. 

To use a custom counter you'll just write an appropriately formatted line to the standard error stream. For example the following line would increment a counter called 'counter-name' in group 'MyWordCounters' by 10:
 > `sys.stderr.write("reporter:counter:MyWordCounters,counter-name,10\n")`
 
This line can be added to your mapper, combiner or reducer scripts and wraped in `if` clauses to increment only under certain conditions. If a counter with that name/group doesn't exsit yet the framework will create one. The values of your custom counters will be printed to the console in their respective groups just like the built in Job Tracking counters. Counters can only be incremented using integers (ie, floats are not supported).

Ok, armed with this new tool let's return to the relative frequencies task.

__DISCUSSION:__
> * Remember the problem we encountered at the end of Exercise 1? How would a custom partition key help us make sure the total counts get sent to each reducer node?
> * What does this do to the number of records being shuffled?
> * Where would you implement the custom partition key?
> * How would you partition the records? (i.e what criteria would you use to assign keys)

### Exercise 3 Tasks:

* __a) add a custom partitioner:__  In __`MultiPart/mapper.py`__, __`MultiPart/combiner.py`__, and __`MultiPart/reducer.py`__ we've copied the base code from the frequencies job. Complete the code in this mapper so that it adds a custom partition key to each record and emits the total subcounts _once for each partition_. Assume we'll be using 4 partitions. Then make any required adjustments to your reducer and combiner to accomodate the new record format (your final output should not include the partition key, though you may want it there initially for debugging purposes).

* __b) discuss:__ Keep in mind that each partition still needs the total counts to arrive before the individual word counts. However since you've added a custom partition key to each record, the order inversion trick of adding a special character isn't going to work with Hadoop's default sorting behavior any more. Why not? What will you need to specify in your Hadoop job to make sure that the totals still arrive first?

* __c) unit test & run your job:__ Write a few unit tests to debug your scripts. When you have them working as expected run the Hadoop job. [__`NOTE`__ We've provided a few tests to get you started but you'll need to add a unix sort to mimic the sorting you discussed in part 'b'].

* __d) custom counters:__ Add custom counter(s) to your reducer code so that you can count how many records are processed by each reduce task. [__`TIP`__ use the partition key as the counter name and python3 string formatting to do this efficiently].

* __e) discuss:__ Rerun your Hadoop job and take a look at the custom counter values. What do they tell you? Are these counts the same as the number of keys in the result of each partition? Try changing the partitioning criteria in your mapper... what partitioning results in a really uneven split? what partitioning results in the most even split?

In [ ]:
# part c - make sure scripts are executable (RUN THIS CELL AS IS)
!chmod a+x MultiPart/mapper.py
!chmod a+x MultiPart/combiner.py
!chmod a+x MultiPart/reducer.py

In [ ]:
# part c - unit test mapper script
!echo "foo foo quux labs foo bar quux" | MultiPart/mapper.py

In [ ]:
# part c - unit test map-combine (ADJUST SORT AS NEEDED)
!echo "foo foo quux labs foo bar quux" | MultiPart/mapper.py | sort -k1,1 | MultiPart/combiner.py

In [ ]:
# part c - unit test map-combine-reduce (ADJUST SORT AS NEEDED)
!echo "foo foo quux labs foo bar quux" | MultiPart/mapper.py | sort -k1,1 | MultiPart/combiner.py | MultiPart/reducer.py 

In [ ]:
# parts c - clear the output directory (RUN THIS CELL AS IS)
!hdfs dfs -rm -r {HDFS_DIR}/multipart-output
# NOTE: this directory won't exist unless you are re-running a job, that's fine.

In [ ]:
# part c - Hadoop streaming command - FILL IN HERE (don't forget to specify 4 reducers)







In [ ]:
#  part c - take a look at a few records from each partition (RUN THIS CELL AS IS)
for p in range(4):
    print('='*10,f'PARTITION{p+1}','='*10)
    !hdfs dfs -cat {HDFS_DIR}/multipart-output/part-0000{p} | head -n 5

__Expected Results:__  
__`NOTE:`__ exact partition splits depend on your custom function:

Test file:
<table>
<th>partition 1</th>
<th>partition 2</th>
<th>partition 3</th>
<th>partition 4</th>
<tr>
<td><pre>
test	0.15
this	0.15
two	0.05
</pre></td>
<td><pre>
a	0.1
file	0.15
for	0.05
</pre></td>
<td><pre>
has	0.05
is	0.1
lines	0.05
</pre></td>
<td><pre>
small	0.15
</pre></td>
</tr></table>

Full Alice Text:
<table>
<th>partition 1</th>
<th>partition 2</th>
<th>partition 3</th>
<th>partition 4</th>
<tr>
<td><pre>
t	0.00716563
table	0.00059165
tail	0.00029582
tails	9.8609e-05
take	0.00072313
</pre></td>
<td><pre>
a	0.022680209
abide	6.5739e-05
about	0.00174210
able	3.2869e-05
above	3.2869e-05
</pre></td>
<td><pre>
gained	3.2869e-05
gallons	3.2869e-05
game	0.00042730
games	3.2869e-05
garden	0.00052591
</pre></td>
<td><pre>
name	0.00036156
named	3.2869e-05
names	6.5739e-05
narrow	6.5739e-05
natural	3.2869e-05
</pre></td>
</tr></table>

>__DISCUSSION__
> * After adding the custom partition key what else did you have to do so that the order inversion pattern would still work?
> * In part `d` what did you see from your custom counters? Do these numbers match the number of keys in the result of each partition? Why/why not?
> * What other partitioning cuts did you try? What partitioning results in a really uneven split? what partitioning results in the most even split?
> * Do custom counters solve the load balancing challenge?
> * If we wanted to design a subsequent job to sort these results from highest to lowest frequency what partitioning strategy would you explore? Any particular challenge there? (__`HINT:`__ you may wish to consider the kinds of numbers you saw in the results from Exercise 1, how python stores floats, and what you know about word frequencies in natural language).

# Exercise 4: Unordered Total Sort vs Total Order Sort

In HW1 we emphasized that sorting can be a useful preprocessing tool because sorted input can sometimes facilitate a more efficient algorithm design. However sorting is also often desirable in its own right. For example suppose you wanted to get the top and bottom 100 words according to their relative frequencies? The best way to get these results would be to do a Total Order Sort -- that is to sort the entire data set from highest to lowest so that you can simply read the first 100 records in the first partition and last 100 records in the last partition.

So far we've tried two different ways of sorting using the Hadoop framework and neither quite achieved this result to satisfaction. First, in last week's breakout we performed a secondary sort using the `keycomparator` and accompanying Hadoop parameters. When we used a single reducer that strategy _did_ successfully result in a sort from top to bottom, but in a class all about scaling up for large data solutions that require a single reducer won't get us very far. Unfortunately when we tried using those same three sorting parameters and with multiple reducers we ended up with results sorted within each partition but not across partitions. This is what we call that a _partial sort_ because records are only ordered relative to the other keys that happen to be processed by the same reducer. Importantly a partial sort is of no use at all for us if we wanted to find the top 100 and bottom 100 from our relative frequencies file.

Then in Exercise 2 of this notebook we learned how to specify a custom partition key so that we can explicitly control the partitioning. By combining the use of a custom partition key and the secondary sort from last week we were able to ensure that our results were not only sorted within their paritions but also that the paritioning grouped records with similar value ranges together. In theory this should have solved our 'total sort with multiple reducers' challenge but in practice something odd happened: the records with the top value didn't reliably end up in the first partition nor did the records with the lowest values necessarily end up in the last. The partition files themselves were out of order, making this an _unordered total sort_.

The reason that the partitions appear out of order has to do with the hash function that Hadoop applies to your custom partition keys. That hash function results in an ordering that may not match the human readible string or integer numbering you as a programmer might have intended. Luckily, this hash function is both fixed and known -- so for example, if you are using partition keys `A`, `B`, and `C` your partitions will always end up ordered `B` - `C` -`A` (as you saw in Exercise 2). That means if we know in advance how many partitions (i.e. reducers) we plan to use, we can reverse engineer the hash function to figure out how Hadoop will order those keys and plan accordingly (for example in our 3 partition case by specifying that the top values should get the partition key `B` not `A`). When your data is sorted not only within each partition but the partitions are also in the right order, you've achieved a _total order sort_. 

Let's give it a try! Your job in this exercise is to write a Hadoop Job that will perform a total order sort on the output of Exercise 1 (relative frequencies) with any number of partitions.

__Note__: Part III.D.4 in the [Total Sort Notebook](https://github.com/UCB-w261/main/blob/master/HelpfulResources/TotalSortGuide/_total-sort-guide-spark2.01-JAN27-2017.ipynb) has a much more comprehensive explanation that you may wish to reference here (ignore the MRJob code).

/user/root/demo3/frequencies-output

### Exercise 4 Tasks
* __a) discuss:__ In the `%%writefile` cell below we create a partition file (__`partitions.txt`__) which contains the cut points we'll use to partition the data. Based on your reading of this file, how many partitions will we use? Think about the kinds of values that we got in Exercise 1 ... can you see any potential problems with these cutpoints?


* __b) discuss:__ Read through __`TotalOrderSort/mapper.py`__ and __`TotalOrderSort/reducer.py`__. Which of these scripts makes use of our partition file? What does the mapper do? What does the reducer do? 

* __c) code:__ Run the provided code below to apply this mapper and reducer. Note how the Hadoop job reads directly from the output directory we created in Exercise 1 (you will need to be those results were computed inorder to run this job). Use the provided code to view the output and confirm that the current implementation performs an unordered sort. What adjustments might you want to make to the partition file? (go ahead and modify it as desired). Then modify the Hadoop job so that it uses `/bin/cat` instead of `reducer.py`... and re-run the job this will allow you to see the order in which the partition keys were sorted. 

* __d) discuss:__ Read through __`TotalOrderSort/TOS_mapper.py`__. Pay particular attention to the helper function `makeKeyHash()` -- what does this function do? how is it used? how is this mapper different that the original one? 

* __e) code + discussion:__ Replace the original mapper with this new mapper(_don't forget to add it to the `-files` line_) and rerun the job (still with `/bin/cat` as reducer). How did the parition ordering change? What happpens if you change the partition file so that it has one extra number? Does your job work? Re-place the true reducer. Et voila, you have achieved total order sort!

In [ ]:
%%writefile TotalOrderSort/partitions.txt
0.01,0.005,0.003,0.002,0.001,0

In [ ]:
# part c - make sure files are executable (RUN THIS CELL AS IS)
!chmod a+x TotalOrderSort/mapper.py
!chmod a+x TotalOrderSort/reducer.py
!chmod a+x TotalOrderSort/TOS_mapper.py

In [ ]:
# part c - hadoop job (RUN THIS CELL AS IS)
!hdfs dfs -rm -r {HDFS_DIR}/tos-output
!hadoop jar {JAR_FILE} \
  -D stream.num.map.output.key.fields=3 \
  -D mapreduce.job.output.key.comparator.class=org.apache.hadoop.mapred.lib.KeyFieldBasedComparator \
  -D mapreduce.partition.keypartitioner.options="-k1,1" \
  -D mapreduce.partition.keycomparator.options="-k3,3nr" \
  -files TotalOrderSort/mapper.py,TotalOrderSort/reducer.py,TotalOrderSort/partitions.txt \
  -mapper mapper.py \
  -reducer reducer.py \
  -partitioner org.apache.hadoop.mapred.lib.KeyFieldBasedPartitioner \
  -input {HDFS_DIR}/frequencies-output \
  -output {HDFS_DIR}/tos-output \
  -cmdenv PATH={PATH} \
  -numReduceTasks 6

In [ ]:
# part c - print top words in each class (RUN THIS CELL AS IS)
for idx in range(6):
    print(f"\n============== PART-0000{idx}===============")
    numLines = !hdfs dfs -cat {HDFS_DIR}/tos-output/part-0000{idx} | wc -l
    print(f"Number of lines processed by this reducer: {numLines[0]}\n")
    !hdfs dfs -cat {HDFS_DIR}/tos-output/part-0000{idx} | head | column -t